# Exploring ONC Data Web Services

Exploring the ONC data web services documented at https://wiki.oceannetworks.ca/display/help/API

In [1]:
import json
import os
from urllib.parse import quote, urlencode

import arrow
import requests

The web service URLS are composed of:

* the base URL: `http://dmas.uvic.ca/api/
* an end-point: presently one of `archivefiles`, `dataproducts`, `rawdata`, `scalardata`, `stations`, or `status`
* a query string that includes `method=methodName`, `token=USER_TOKEN`, and other end-point-specific key/value pairs

For example:

  `http://dmas.uvic.ca/api/scalardata?method=getByStation&token=<yourValidToken>&station=SCHDW.O1&deviceCategory=OXYSENSOR`

Access to the web services requires a user token which you can generate on the 
`Web Services API` tab of your [ONC account profile page](http://dmas.uvic.ca/Profile).
I have stored mine in an environment variable so as not to publish it to the world
in this notebook.

In [2]:
TOKEN = os.environ['ONC_USER_TOKEN']

Here's a string template for a request URL, and a technique for composing the query string:

In [3]:
url_tmpl = 'http://dmas.uvic.ca/api/{endpoint}?{query}'

In [4]:
url_tmpl.format(
    endpoint='scalardata',
    query=urlencode({
        'method': 'getByStation',
        'token': 'USER_TOKEN',
        'station': 'SCHDW.O1',
        'deviceCategory': 'OXYSENSOR',
    }, quote_via=quote, safe='/:'))

'http://dmas.uvic.ca/api/scalardata?token=USER_TOKEN&deviceCategory=OXYSENSOR&station=SCHDW.O1&method=getByStation'

Substituting my real token,
and using the `requests` package to send the data request to the web service:

In [5]:
data_url = url_tmpl.format(
    endpoint='scalardata',
    query=urlencode({
        'method': 'getByStation',
        'token': TOKEN,
        'station': 'SCHDW.O1',
        'deviceCategory': 'OXYSENSOR',
    }, quote_via=quote, safe='/:'))

response = requests.get(data_url)
response.raise_for_status()

Calling the `raise_for_status()` method on the response object is a quick
way to test for HTTP errors.

The default response type from the web services is JSON
and `requests` provides a convenience method to convert JSON
in the response to a Python `dict` object:

In [6]:
response.json()

{'sensorData': [{'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-05-02T20:13:10.064Z',
     'value': 2.093462278}],
   'sensor': 'oxygen',
   'sensorName': 'Oxygen',
   'unitOfMeasure': 'ml/l'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-05-02T20:13:10.064Z',
     'value': 9.9706}],
   'sensor': 'temperature',
   'sensorName': 'Temperature',
   'unitOfMeasure': 'C'}],
 'serviceMetadata': {'dateFrom': None,
  'dateTo': None,
  'deviceCategory': 'OXYSENSOR',
  'nextDateFrom': None,
  'outputFormat': None,
  'rowLimit': None,
  'sensors': None,
  'station': 'SCHDW.O1',
  'totalActualSamples': 2}}

So, let's put that all together into a function to query the ONC data web services:

In [7]:
def get_onc_data(endpoint, method, token, **query_params):
    url_tmpl = 'http://dmas.uvic.ca/api/{endpoint}?{query}'
    query = {'method': method, 'token': token}
    query.update(query_params)
    print(query)
    data_url = url_tmpl.format(
        endpoint=endpoint,
        query=urlencode(query, quote_via=quote, safe='/:'))
    print(data_url)
    response = requests.get(data_url)
    response.raise_for_status()
    return response.json()

In [8]:
get_onc_data('scalardata', 'getByStation', TOKEN, station='SCHDW.O1', deviceCategory='OXYSENSOR')

{'token': '8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d', 'deviceCategory': 'OXYSENSOR', 'station': 'SCHDW.O1', 'method': 'getByStation'}
http://dmas.uvic.ca/api/scalardata?token=8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d&deviceCategory=OXYSENSOR&station=SCHDW.O1&method=getByStation


{'sensorData': [{'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-05-02T20:13:10.064Z',
     'value': 2.093462278}],
   'sensor': 'oxygen',
   'sensorName': 'Oxygen',
   'unitOfMeasure': 'ml/l'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-05-02T20:13:10.064Z',
     'value': 9.9706}],
   'sensor': 'temperature',
   'sensorName': 'Temperature',
   'unitOfMeasure': 'C'}],
 'serviceMetadata': {'dateFrom': None,
  'dateTo': None,
  'deviceCategory': 'OXYSENSOR',
  'nextDateFrom': None,
  'outputFormat': None,
  'rowLimit': None,
  'sensors': None,
  'station': 'SCHDW.O1',
  'totalActualSamples': 2}}

## `stations` End-point

The [`stations` end-point](https://wiki.oceannetworks.ca/display/help/stations)
has a `getTree` method that returns a large, hierarchical tree data structure of
stations, station codes, and devices.
```python
get_onc_data('stations', 'getTree', TOKEN)
```
Rather than reading or parsing the data structure here,
it is perhaps easier to find station codes and device categories to use with the
other end-points by looking at the list generated by the
[Javascript Usage Exmaple](https://wiki.oceannetworks.ca/download/attachments/42172426/stationTreeExample.html?version=2&modificationDate=1428438144000&api=v2)
provided by ONC.

## `scalardata` End-point

The [`scalardata` end-point](https://wiki.oceannetworks.ca/display/help/scalardata+service)
has a `getByStation` method that returns time series of data given a station code
and an device category code.
Its simplest use case is to return the most recent data for all sensors associated with
the device category at the station.

### VENUS Node CTD Data

Here that is for the CTD at the Salish Sea Central node VENUS Instrument Platform:

In [9]:
get_onc_data('scalardata', 'getByStation', TOKEN, station='SCVIP', deviceCategory='CTD')

{'token': '8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d', 'deviceCategory': 'CTD', 'station': 'SCVIP', 'method': 'getByStation'}
http://dmas.uvic.ca/api/scalardata?token=8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d&deviceCategory=CTD&station=SCVIP&method=getByStation


{'sensorData': [{'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T22:27:50.509Z',
     'value': 23.98920213}],
   'sensor': 'SIGMA_THETA',
   'sensorName': 'Sigma-theta (0 dbar)',
   'unitOfMeasure': 'kg/m3'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T22:27:50.509Z',
     'value': 3.38175}],
   'sensor': 'cond',
   'sensorName': 'Conductivity',
   'unitOfMeasure': 'S/m'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T22:27:50.509Z',
     'value': 1025.340066607}],
   'sensor': 'density',
   'sensorName': 'Density',
   'unitOfMeasure': 'kg/m3'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T22:27:51.509Z',
     'value': 297.979}],
   'sensor': 'pressure',
   'sensorName': 'Pressure',
   'unitOfMeasure': 'decibar'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T22:27:50.509Z',
     'value': 31.0707}],
   'sensor'

Adding a `sensors` item to the query with a value that is a comma-separated list
of `sensor` codes limits the response to contain only the data from the specified sensors:

In [10]:
get_onc_data('scalardata', 'getByStation', TOKEN, station='SCVIP', deviceCategory='CTD', sensors='salinity,temperature')

{'sensors': 'salinity,temperature', 'token': '8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d', 'deviceCategory': 'CTD', 'station': 'SCVIP', 'method': 'getByStation'}
http://dmas.uvic.ca/api/scalardata?sensors=salinity%2Ctemperature&token=8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d&deviceCategory=CTD&station=SCVIP&method=getByStation


{'sensorData': [{'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T22:27:51.509Z',
     'value': 31.0707}],
   'sensor': 'salinity',
   'sensorName': 'Practical Salinity',
   'unitOfMeasure': 'psu'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T22:27:51.509Z',
     'value': 9.3807}],
   'sensor': 'temperature',
   'sensorName': 'Temperature',
   'unitOfMeasure': 'C'}],
 'serviceMetadata': {'dateFrom': None,
  'dateTo': None,
  'deviceCategory': 'CTD',
  'nextDateFrom': None,
  'outputFormat': None,
  'rowLimit': None,
  'sensors': 'salinity,temperature',
  'station': 'SCVIP',
  'totalActualSamples': 2}}

Time series of data are obtained by adding `dateFrom` and `dateTo` items to the query:

In [11]:
get_onc_data(
    'scalardata', 'getByStation', TOKEN, station='SCVIP', deviceCategory='CTD',
    sensors='salinity,temperature',
    dateFrom='2016-06-21T19:03:45.000Z',
)

{'token': '8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d', 'dateFrom': '2016-06-21T19:03:45.000Z', 'station': 'SCVIP', 'deviceCategory': 'CTD', 'sensors': 'salinity,temperature', 'method': 'getByStation'}
http://dmas.uvic.ca/api/scalardata?token=8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d&dateFrom=2016-06-21T19:03:45.000Z&station=SCVIP&deviceCategory=CTD&sensors=salinity%2Ctemperature&method=getByStation


{'sensorData': [{'actualSamples': 12247,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T19:03:45.347Z',
     'value': 31.1534},
    {'qaqcFlag': 1, 'sampleTime': '2016-06-21T19:03:46.359Z', 'value': 31.154},
    {'qaqcFlag': 1,
     'sampleTime': '2016-06-21T19:03:47.348Z',
     'value': 31.1545},
    {'qaqcFlag': 1, 'sampleTime': '2016-06-21T19:03:48.356Z', 'value': 31.155},
    {'qaqcFlag': 1,
     'sampleTime': '2016-06-21T19:03:49.347Z',
     'value': 31.1554},
    {'qaqcFlag': 1,
     'sampleTime': '2016-06-21T19:03:50.363Z',
     'value': 31.1559},
    {'qaqcFlag': 1,
     'sampleTime': '2016-06-21T19:03:51.347Z',
     'value': 31.1552},
    {'qaqcFlag': 1,
     'sampleTime': '2016-06-21T19:03:52.358Z',
     'value': 31.1555},
    {'qaqcFlag': 1,
     'sampleTime': '2016-06-21T19:03:53.347Z',
     'value': 31.1549},
    {'qaqcFlag': 1,
     'sampleTime': '2016-06-21T19:03:54.363Z',
     'value': 31.1525},
    {'qaqcFlag': 1,
     'sampleTime': '2016-06-21T19:03:55.348

With only a `dateFrom` item in the query the time series length defaults to 1 day in length.
The `dateTo` query item controls the length of the time series by date/time stamp.

In [12]:
get_onc_data(
    'scalardata', 'getByStation', TOKEN, station='SCVIP', deviceCategory='CTD',
    sensors='salinity,temperature',
    dateFrom='2016-06-21T17:58:45.000Z', dateTo='2016-06-21T17:58:50.000Z',
)

{'token': '8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d', 'dateFrom': '2016-06-21T17:58:45.000Z', 'dateTo': '2016-06-21T17:58:50.000Z', 'station': 'SCVIP', 'deviceCategory': 'CTD', 'sensors': 'salinity,temperature', 'method': 'getByStation'}
http://dmas.uvic.ca/api/scalardata?token=8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d&dateFrom=2016-06-21T17:58:45.000Z&dateTo=2016-06-21T17:58:50.000Z&station=SCVIP&deviceCategory=CTD&sensors=salinity%2Ctemperature&method=getByStation


{'sensorData': [{'actualSamples': 5,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T17:58:45.400Z',
     'value': 31.1297},
    {'qaqcFlag': 1,
     'sampleTime': '2016-06-21T17:58:46.440Z',
     'value': 31.1294},
    {'qaqcFlag': 1,
     'sampleTime': '2016-06-21T17:58:47.400Z',
     'value': 31.1285},
    {'qaqcFlag': 1,
     'sampleTime': '2016-06-21T17:58:48.444Z',
     'value': 31.1285},
    {'qaqcFlag': 1,
     'sampleTime': '2016-06-21T17:58:49.400Z',
     'value': 31.1282}],
   'sensor': 'salinity',
   'sensorName': 'Practical Salinity',
   'unitOfMeasure': 'psu'},
  {'actualSamples': 5,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T17:58:45.400Z',
     'value': 9.3081},
    {'qaqcFlag': 1, 'sampleTime': '2016-06-21T17:58:46.440Z', 'value': 9.3083},
    {'qaqcFlag': 1, 'sampleTime': '2016-06-21T17:58:47.400Z', 'value': 9.3087},
    {'qaqcFlag': 1, 'sampleTime': '2016-06-21T17:58:48.444Z', 'value': 9.3088},
    {'qaqcFlag': 1, 'sampleTime': '2016-06-21T

The number of measurements returned can be specified directly with the `rowLimit` query item.
*Note also that there is a hard limit of 100,000 measurements per sensor per request.*

In [13]:
get_onc_data(
    'scalardata', 'getByStation', TOKEN, station='SCVIP', deviceCategory='CTD',
    sensors='salinity,temperature',
    dateFrom='2016-06-21T17:58:45.000Z', rowLimit=2,
)

{'token': '8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d', 'dateFrom': '2016-06-21T17:58:45.000Z', 'rowLimit': 2, 'station': 'SCVIP', 'deviceCategory': 'CTD', 'sensors': 'salinity,temperature', 'method': 'getByStation'}
http://dmas.uvic.ca/api/scalardata?token=8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d&dateFrom=2016-06-21T17:58:45.000Z&rowLimit=2&station=SCVIP&deviceCategory=CTD&sensors=salinity%2Ctemperature&method=getByStation


{'sensorData': [{'actualSamples': 2,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T17:58:45.400Z',
     'value': 31.1297},
    {'qaqcFlag': 1,
     'sampleTime': '2016-06-21T17:58:46.440Z',
     'value': 31.1294}],
   'sensor': 'salinity',
   'sensorName': 'Practical Salinity',
   'unitOfMeasure': 'psu'},
  {'actualSamples': 2,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T17:58:45.400Z',
     'value': 9.3081},
    {'qaqcFlag': 1,
     'sampleTime': '2016-06-21T17:58:46.440Z',
     'value': 9.3083}],
   'sensor': 'temperature',
   'sensorName': 'Temperature',
   'unitOfMeasure': 'C'}],
 'serviceMetadata': {'dateFrom': '2016-06-21T17:58:45.000Z',
  'dateTo': None,
  'deviceCategory': 'CTD',
  'nextDateFrom': '2016-06-21T17:58:47.400Z',
  'outputFormat': None,
  'rowLimit': 2,
  'sensors': 'salinity,temperature',
  'station': 'SCVIP',
  'totalActualSamples': 4}}

The values for `dateFrom` and `dateTo` are in UTC and must be strings formatted
as `yyyy-MM-ddTHH:mm:ss.SSSZ`.
That format is annoying enough to type,
and timezone conversions are error-prone enough
that it is worth writing a function to handle the details:

In [14]:
def onc_datetime(datetime_str, timezone='Canada/Pacific'):
    print(datetime_str)
    d = arrow.get(datetime_str)
    print(d)
    d_tz = arrow.get(d.datetime, timezone)
    print(d_tz)
    d_utc = d_tz.to('utc')
    print(d_utc)
    return '{}Z'.format(d_utc.format('YYYY-MM-DDTHH:mm:ss.SSS'))

In [15]:
onc_datetime('2016-06-21 10:58:45')

2016-06-21 10:58:45
2016-06-21T10:58:45+00:00
2016-06-21T10:58:45-07:00
2016-06-21T17:58:45+00:00


'2016-06-21T17:58:45.000Z'

In [16]:
onc_datetime('2016-06-21 17:58:45', 'utc')

2016-06-21 17:58:45
2016-06-21T17:58:45+00:00
2016-06-21T17:58:45+00:00
2016-06-21T17:58:45+00:00


'2016-06-21T17:58:45.000Z'

In [17]:
get_onc_data(
    'scalardata', 'getByStation', TOKEN, station='SCVIP', deviceCategory='CTD',
    sensors='salinity,temperature',
    dateFrom=onc_datetime('2016-06-21 10:58:45'), dateTo=onc_datetime('2016-06-21 10:58:50'),
)

2016-06-21 10:58:45
2016-06-21T10:58:45+00:00
2016-06-21T10:58:45-07:00
2016-06-21T17:58:45+00:00
2016-06-21 10:58:50
2016-06-21T10:58:50+00:00
2016-06-21T10:58:50-07:00
2016-06-21T17:58:50+00:00
{'token': '8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d', 'dateFrom': '2016-06-21T17:58:45.000Z', 'dateTo': '2016-06-21T17:58:50.000Z', 'station': 'SCVIP', 'deviceCategory': 'CTD', 'sensors': 'salinity,temperature', 'method': 'getByStation'}
http://dmas.uvic.ca/api/scalardata?token=8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d&dateFrom=2016-06-21T17:58:45.000Z&dateTo=2016-06-21T17:58:50.000Z&station=SCVIP&deviceCategory=CTD&sensors=salinity%2Ctemperature&method=getByStation


{'sensorData': [{'actualSamples': 5,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T17:58:45.400Z',
     'value': 31.1297},
    {'qaqcFlag': 1,
     'sampleTime': '2016-06-21T17:58:46.440Z',
     'value': 31.1294},
    {'qaqcFlag': 1,
     'sampleTime': '2016-06-21T17:58:47.400Z',
     'value': 31.1285},
    {'qaqcFlag': 1,
     'sampleTime': '2016-06-21T17:58:48.444Z',
     'value': 31.1285},
    {'qaqcFlag': 1,
     'sampleTime': '2016-06-21T17:58:49.400Z',
     'value': 31.1282}],
   'sensor': 'salinity',
   'sensorName': 'Practical Salinity',
   'unitOfMeasure': 'psu'},
  {'actualSamples': 5,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T17:58:45.400Z',
     'value': 9.3081},
    {'qaqcFlag': 1, 'sampleTime': '2016-06-21T17:58:46.440Z', 'value': 9.3083},
    {'qaqcFlag': 1, 'sampleTime': '2016-06-21T17:58:47.400Z', 'value': 9.3087},
    {'qaqcFlag': 1, 'sampleTime': '2016-06-21T17:58:48.444Z', 'value': 9.3088},
    {'qaqcFlag': 1, 'sampleTime': '2016-06-21T

### Ferry Temperature and Salinity Data

The instrumented ferries are stations.
Here is the most recent available data from the TSG device
aboard the Tsawwassen to Duke Point ferry:

In [18]:
get_onc_data('scalardata', 'getByStation', TOKEN, station='TWDP', deviceCategory='TSG')

{'token': '8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d', 'deviceCategory': 'TSG', 'station': 'TWDP', 'method': 'getByStation'}
http://dmas.uvic.ca/api/scalardata?token=8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d&deviceCategory=TSG&station=TWDP&method=getByStation


{'sensorData': [{'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T05:27:56.148Z',
     'value': 2.88854}],
   'sensor': 'Conductivity',
   'sensorName': 'Conductivity',
   'unitOfMeasure': 'S/m'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T05:27:56.148Z',
     'value': 23.3721}],
   'sensor': 'salinity',
   'sensorName': 'Practical Salinity',
   'unitOfMeasure': 'psu'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T05:27:56.148Z',
     'value': 13.628}],
   'sensor': 'temperature',
   'sensorName': 'Temperature',
   'unitOfMeasure': 'C'}],
 'serviceMetadata': {'dateFrom': None,
  'dateTo': None,
  'deviceCategory': 'TSG',
  'nextDateFrom': None,
  'outputFormat': None,
  'rowLimit': None,
  'sensors': None,
  'station': 'TWDP',
  'totalActualSamples': 3}}

Note that the data may lag the present time by several hours because it is only
transmitted from the ferry to the ONC servers when the ferry is at dock.
Also, there appears to be a several hours long gap in the data each day,
presumably while the ferry is docked overnight.

The ferry's location is available from the NAV device.
*Note that the times from the TSG and NAV devices do not appear to be synchronized.*

In [19]:
get_onc_data('scalardata', 'getByStation', TOKEN, station='TWDP', deviceCategory='NAV')

{'token': '8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d', 'deviceCategory': 'NAV', 'station': 'TWDP', 'method': 'getByStation'}
http://dmas.uvic.ca/api/scalardata?token=8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d&deviceCategory=NAV&station=TWDP&method=getByStation


{'sensorData': [{'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T05:17:53.196Z',
     'value': 49.0057}],
   'sensor': 'Latitude',
   'sensorName': 'Latitude',
   'unitOfMeasure': 'deg'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T05:17:53.196Z',
     'value': -123.1295}],
   'sensor': 'Longitude',
   'sensorName': 'Longitude',
   'unitOfMeasure': 'deg'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T05:17:53.196Z',
     'value': 3.0}],
   'sensor': 'ship_course',
   'sensorName': 'Ship Course',
   'unitOfMeasure': 'deg'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T05:17:53.196Z',
     'value': 0.0}],
   'sensor': 'speed_over_ground',
   'sensorName': 'Speed over Ground',
   'unitOfMeasure': 'm/s'}],
 'serviceMetadata': {'dateFrom': None,
  'dateTo': None,
  'deviceCategory': 'NAV',
  'nextDateFrom': None,
  'outputFormat': None,
  'rowLimit':

Just fiddling around from here on....

In [20]:
get_onc_data(
    'scalardata', 'getByStation', TOKEN, station='BDYPM', deviceCategory='CTD',
)

{'token': '8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d', 'deviceCategory': 'CTD', 'station': 'BDYPM', 'method': 'getByStation'}
http://dmas.uvic.ca/api/scalardata?token=8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d&deviceCategory=CTD&station=BDYPM&method=getByStation


{'sensorData': [{'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2015-04-09T14:50:31.000Z',
     'value': 3.32322}],
   'sensor': 'cond',
   'sensorName': 'Conductivity',
   'unitOfMeasure': 'S/m'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2015-04-09T14:50:31.000Z',
     'value': 1025.029060561}],
   'sensor': 'density',
   'sensorName': 'Density',
   'unitOfMeasure': 'kg/m3'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2015-04-09T14:50:31.000Z',
     'value': 218.533}],
   'sensor': 'pressure',
   'sensorName': 'Pressure',
   'unitOfMeasure': 'decibar'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2015-04-09T14:50:31.000Z',
     'value': 31.0154}],
   'sensor': 'salinity',
   'sensorName': 'Practical Salinity',
   'unitOfMeasure': 'psu'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2015-04-09T14:50:31.000Z',
     'value': 24.031170169}],
   'sensor': 'sig

In [21]:
get_onc_data('scalardata', 'getByStation', TOKEN, station='SCVIP', deviceCategory='ADCP150KHZ')

{'token': '8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d', 'deviceCategory': 'ADCP150KHZ', 'station': 'SCVIP', 'method': 'getByStation'}
http://dmas.uvic.ca/api/scalardata?token=8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d&deviceCategory=ADCP150KHZ&station=SCVIP&method=getByStation


{'sensorData': [{'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T22:27:55.805Z',
     'value': 273.85}],
   'sensor': 'heading',
   'sensorName': 'Magnetic Compass Heading',
   'unitOfMeasure': 'deg'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T22:27:55.805Z',
     'value': 0.81}],
   'sensor': 'pitch',
   'sensorName': 'Pitch',
   'unitOfMeasure': 'deg'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T22:27:55.805Z',
     'value': 295.222}],
   'sensor': 'pressure',
   'sensorName': 'Pressure',
   'unitOfMeasure': 'decibar'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 1,
     'sampleTime': '2016-06-21T22:27:55.805Z',
     'value': -0.26}],
   'sensor': 'roll',
   'sensorName': 'Roll',
   'unitOfMeasure': 'deg'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 0,
     'sampleTime': '2016-06-21T22:27:55.805Z',
     'value': 1488.0}],
   'sensor': 'sound_speed',
   'sensorName': 

In [22]:
get_onc_data('scalardata', 'getByStation', TOKEN, station='SCVIP', deviceCategory='CURRENTMETER')

{'token': '8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d', 'deviceCategory': 'CURRENTMETER', 'station': 'SCVIP', 'method': 'getByStation'}
http://dmas.uvic.ca/api/scalardata?token=8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d&deviceCategory=CURRENTMETER&station=SCVIP&method=getByStation


{'sensorData': [{'actualSamples': 1,
   'data': [{'qaqcFlag': 0,
     'sampleTime': '2016-06-21T22:27:57.700Z',
     'value': 121.0}],
   'sensor': 'Amplitude_Beam_1',
   'sensorName': 'Amplitude (Beam 1)',
   'unitOfMeasure': 'Count'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 0,
     'sampleTime': '2016-06-21T22:27:57.700Z',
     'value': 119.0}],
   'sensor': 'Amplitude_Beam_2',
   'sensorName': 'Amplitude (Beam 2)',
   'unitOfMeasure': 'Count'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 0,
     'sampleTime': '2016-06-21T22:27:57.700Z',
     'value': 124.0}],
   'sensor': 'Amplitude_Beam_3',
   'sensorName': 'Amplitude (Beam 3)',
   'unitOfMeasure': 'Count'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 0,
     'sampleTime': '2016-06-21T22:27:57.700Z',
     'value': 99.0}],
   'sensor': 'Correlation_Beam_1',
   'sensorName': 'Correlation Beam 1',
   'unitOfMeasure': '%'},
  {'actualSamples': 1,
   'data': [{'qaqcFlag': 0,
     'sampleTime': '2016-06-21T22:27:57.700Z',

In [28]:
get_onc_data(
    'archivefiles', 'getList', TOKEN, station='SCVIP', deviceCategory='ADCP150KHZ',
    dateFrom=onc_datetime('2016-06-15'), returnOptions='all',
)

2016-06-15
2016-06-15T00:00:00+00:00
2016-06-15T00:00:00-07:00
2016-06-15T07:00:00+00:00
{'token': '8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d', 'dateFrom': '2016-06-15T07:00:00.000Z', 'returnOptions': 'all', 'station': 'SCVIP', 'deviceCategory': 'ADCP150KHZ', 'method': 'getList'}
http://dmas.uvic.ca/api/archivefiles?token=8efe44eb-0fb5-4413-bba3-ae3dbe0fbb1d&dateFrom=2016-06-15T07:00:00.000Z&returnOptions=all&station=SCVIP&deviceCategory=ADCP150KHZ&method=getList


[{'archivedDate': '2016-06-16T14:39:10.000Z',
  'compression': 'gz',
  'dataProductFormatId': 5,
  'dateFrom': '2016-06-15T00:00:00.000Z',
  'dateTo': '2016-06-16T00:00:00.000Z',
  'deviceId': 65,
  'fileSize': 38873331,
  'filename': 'RDIADCP150WH8580_20160615T000000.000Z.txt',
  'modifyDate': '2016-06-17T04:07:11.000Z',
  'uncompressedFileSize': 117936000},
 {'archivedDate': '2016-06-17T04:15:24.000Z',
  'compression': None,
  'dataProductFormatId': None,
  'dateFrom': '2016-06-15T00:00:01.000Z',
  'dateTo': '2016-06-15T23:59:59.000Z',
  'deviceId': 65,
  'fileSize': 58492800,
  'filename': 'RDIADCP150WH8580_20160615T000001.636Z.rdi',
  'modifyDate': '2016-06-17T04:15:25.000Z',
  'uncompressedFileSize': 58492800},
 {'archivedDate': '2016-06-17T11:57:13.000Z',
  'compression': 'gz',
  'dataProductFormatId': 5,
  'dateFrom': '2016-06-16T00:00:00.000Z',
  'dateTo': '2016-06-17T00:00:00.000Z',
  'deviceId': 65,
  'fileSize': 38772527,
  'filename': 'RDIADCP150WH8580_20160616T000000.000Z.